#Model training - IACHM


This code is used to train our LSTM based model. We dealt gyroscopical and accelerometer data as time series

### Imports and clone data

In [6]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15109 MiB


In [7]:
#================================== Imports ===================================#
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from google.colab import files
#==============================================================================#

In [8]:
#============================= Clone Git repo =================================#
!git clone https://github.com/matthieu-cervera/IACHM.git
#==============================================================================#

fatal: destination path 'IACHM' already exists and is not an empty directory.


In [9]:
# #============================= Connect to Drive ===============================#
# from google.colab import drive
# drive.mount('/content/gdrive/')
# #==============================================================================#

### Data preprocessing

In [12]:
cd /content/

/content


In [15]:
#================================= Data check =================================#
df = pd.read_csv('gesture2.csv')
df.head(5)
#==============================================================================#

,/accx,/accy,/accz,/gyrox,/gyroy,/gyroz
0,-0.0677,-0.2022,1.5538,-0.4818,-0.0936,0.0404
1,0.0801,-0.2894,2.5115,-0.5870,-0.0462,0.0988
2,0.3076,-0.6156,4.2529,-0.7072,0.1017,-0.1989
3,-0.3166,-0.6446,4.1102,-0.6024,-0.4128,-0.2917
4,-0.5070,-0.3706,2.7059,-1.0266,-0.3352,-0.0801


We need to remove NaN values and labelise the data

In [16]:
#============================= Data organisation =============================#
sequences = []
targets = []      # 0 : nothing // 1 : circle // 2 : horizontal lines

for i in range(1,151):
  df = pd.read_csv('gesture'+str(i)+'.csv')
  df.dropna(inplace= True)
  values = df.values
  sequences.append(values)
  if i<=50:
    targets.append(0)
  elif i<=100:
    targets.append(1)
  else:
    targets.append(2)
#==============================================================================#

We need to have a fixed length for each gesture.  The fixed length is chosen accordingly to our data. Globally we prefer not to loose to much data.  
In our case, we figured out that padding the tensor with zeros is irrelevant so we decided to pad with their last value. 

In [17]:
len_sequences = []
for seq in sequences:
    len_sequences.append(len(seq))
pd.Series(len_sequences).describe()

count    150.000000
mean     127.966667
std        0.180107
min      127.000000
25%      128.000000
50%      128.000000
75%      128.000000
max      128.000000
dtype: float64

In [18]:
#====== Padding the sequence with the values in last row to max length ========#
max_seq_length = 128
processed_data_sequences = []
for seq in sequences :
  last_value = seq[-1]
  rows_to_pad = max_seq_length - len(seq)
  if rows_to_pad != 0:
    seq_to_concatenate = np.repeat(last_value,rows_to_pad)
    seq_to_concatenate_reshaped = np.array([seq_to_concatenate])
    padded_seq = np.concatenate([seq,seq_to_concatenate_reshaped])
    processed_data_sequences.append(padded_seq)
  else:
    processed_data_sequences.append(seq)

processed_data_sequences = np.stack(processed_data_sequences)
#==============================================================================#

#==================== Truncate the sequence to length 128 =====================#
seq_length = 128
for seq,i in enumerate(processed_data_sequences):
  processed_data_sequences[i] = seq[:seq_length]
#==============================================================================#

# Create data with target values
data_w_targets = np.array([[processed_data_sequences[i],targets[i]] for i in range(len(processed_data_sequences))])

#================== Shuffling and separating our data =========================#
# We chose 80% train 20% val
threshold = int(0.8*len(data_w_targets)) 
np.random.shuffle(data_w_targets)
train_data = data_w_targets[:threshold]
val_data = data_w_targets[threshold:]
#==============================================================================#

<ipython-input-18-d60ffaaec5d1>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_w_targets = np.array([[processed_data_sequences[i],targets[i]] for i in range(len(processed_data_sequences))])


### Our LSTM based model

In [28]:
input_size = 6 
hidden_size = 128

class LSTMModel(nn.Module):
    def __init__(self, device):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(
            input_size=6, hidden_size=128, batch_first=True)
        self.lstm = nn.LSTM(
            input_size=128, hidden_size=128,batch_first=True, dropout = 0.2)
        self.linear = nn.Linear(128, 3)
        self.sigmoid = nn.Sigmoid()
        self.device = device

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, (ht, ct) = self.lstm(x)
        x = self.linear(ht[-1])
        x = self.sigmoid(x)
        return x

### Validation and training functions




In [20]:
#============================== Val function ================================#
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for row in val_data:
      data = torch.from_numpy(row[0]).float()
      target = torch.from_numpy(np.array(row[1])).float()
      if use_cuda:
            data, target = data.cuda(), target.cuda()
      output = model(data)
      target = target.long()
      criterion = torch.nn.CrossEntropyLoss(reduction='mean')
      validation_loss += criterion(output, target).data.item()
       
      if torch.argmax(output) == row[1]:    
        correct += 1

    validation_loss /= len(val_data)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_data),
        100. * correct / len(val_data)))
#==============================================================================#

In [21]:
#============================== Train function ================================#
def train(epoch, model, optimizer):
    model.train()
    for e in range(1,epoch+1):
      for row in train_data :
        data = torch.from_numpy(row[0]).float()
        target = torch.from_numpy(np.array(row[1])).float()
        if use_cuda:
              data, target = data.cuda(), target.cuda() 
        target = target.long()

        #def closure():
        optimizer.zero_grad()
        output = model(data)
    
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        loss = criterion(output, target)
        loss.backward()
          #return loss
        optimizer.step()
        # optimizer.step(closure)
      # loss = closure()
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
      e,  len(train_data), len(train_data),
      100. / len(train_data), loss.data.item()))
      validation()
      model.train()
#==============================================================================#

### Training and saving the model

In [30]:
use_cuda = torch.cuda.is_available()
torch.cuda.empty_cache()

model = LSTMModel(device='cuda')

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# torch.backends.cuda.matmul.allow_tf32 = False
optimizer = optim.Adam(model.parameters(), lr=0.0001)
nb_epoch = 20

train(nb_epoch,model,optimizer)

Using GPU
Train Epoch: 1 [120/120 (1%)]	Loss: 1.087075

Validation set: Average loss: 1.0961, Accuracy: 12/30 (40%)
Train Epoch: 2 [120/120 (1%)]	Loss: 1.077465

Validation set: Average loss: 1.0796, Accuracy: 12/30 (40%)
Train Epoch: 3 [120/120 (1%)]	Loss: 1.052749

Validation set: Average loss: 1.0396, Accuracy: 12/30 (40%)
Train Epoch: 4 [120/120 (1%)]	Loss: 1.019770

Validation set: Average loss: 0.9988, Accuracy: 16/30 (53%)
Train Epoch: 5 [120/120 (1%)]	Loss: 0.942408

Validation set: Average loss: 0.9063, Accuracy: 21/30 (70%)
Train Epoch: 6 [120/120 (1%)]	Loss: 0.867059

Validation set: Average loss: 0.8160, Accuracy: 25/30 (83%)
Train Epoch: 7 [120/120 (1%)]	Loss: 0.802550

Validation set: Average loss: 0.7573, Accuracy: 26/30 (87%)
Train Epoch: 8 [120/120 (1%)]	Loss: 0.749088

Validation set: Average loss: 0.7180, Accuracy: 29/30 (97%)
Train Epoch: 9 [120/120 (1%)]	Loss: 0.705551

Validation set: Average loss: 0.7068, Accuracy: 26/30 (87%)
Train Epoch: 10 [120/120 (1%)]	Loss:

In [31]:
torch.save(model.state_dict(), 'checkpoint.pth')

# download checkpoint file
files.download('checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>